<img src="../DATA/corp_logo.jpg" width='160'>

# Optical Flow

>#### Note: 
>**If you are a notebook user it is prefered to shutdown other notebooks' kernels and restart the kernel in this notebook. That is because the tracking algorihtm can sometimes get caught in a loop with your camera.**

OPtical flow is the pattern of apparent motion of image objects between two consecutive frames caused by the movement of object or camera (caused by the either movement of objects or camera)

## Assumptions:
- The pixel intensities of an object do not change between consecutive frames. (We are not trying to track a lightbulb that is turning off and on as it moves between frames of a video.)
- Neibouring pixels have similar motions. (The actual pixels around the point you're trying to track such as the center of light bulb are also moving along). So it's not just some random single point moving.


>**Note:**
<br>- The optical flow methods in OpenCV will first take in a given set of points and a frame.
<br>- then it will attemp to find those points in the next frame.
<br>- It is up to the user to supply the points to track.

<img src="./images/five_frames_of_ball_0.png" width= "500">
The image above, displays a five frame clip of a ball moving up and towards the right. Note that in the optical flow technique, given just this clip, there is no way to determine if the ball is moving, or if the camera moved down and to the left.
<br>
<br>

<img src="./images/five_frames_of_ball_1.png" width= "500">
Using OpenCV we pass in the previous frame, previous points and the current frame to the optical flow function named 
**Lucas-Kanade** function.We are actually trying to track the object from frame 4 to frame 5 (the most recent frame in our video). We would have had to have told Open see what the point we wanted to track (the green point on the frame 4) or more advanced techniques such as object detection).
<br>
<br>

<img src="./images/five_frames_of_ball_2.png" width= "500">
Then we take in the entire previous frame.
<br>
<br>

<img src="./images/five_frames_of_ball_3.png" width= "500">
Then we look at our current frame. 
<br>
<br>

<img src="./images/five_frames_of_ball_4.png" width= "500">
What this function attempts to do is locate the points from frame for that we wanted to track and find them in frame 5.

>**Note:**
<br>- The Lucas-kanade computes optical flow for a **sparse** (meaning only the points it was told to track) feature set. In other words, it is not suitalbe for cases with purpose of tracking all the points in video.

Gunner Farneback's algorithm(also built-in to OpenCV) is used for calculating **dense function** optical flow(calculate flow for all points in an image). It will color them black if no flow(no movement) is detected.
<br>
<br>

---


## Lucas-Kanade Optical Flow

In [1]:
import numpy as np
import cv2 

In [2]:
# Parameters for ShiTomasi corner detection (good features to track paper)
# On the very first frame, detect 10 corners
corner_track_params = {
    'maxCorners': 10,
    'qualityLevel': 0.3,
    'minDistance': 7,
    'blockSize': 7
}

### Parameters for Lucas-Kanade Optical Flow

- **winSize:** Detect the motion of specific points or the aggregated motion of regions by modifying the `winSize` argument. This determines the integration window size. Small windows are more sensitive to noise and may miss larger motions. Large windows will “survive” an occlusion and less able to catch smaller motions. The integration appears smoother with the larger window size.

- **maxLevel:** When `maxLevel` is 0, it is the same algorithm without using pyramids (ie, calcOpticalFlowLK). Pyramids allow finding optical flow at various resolutions of the image. Check out the link for the [Pyramid image processing](https://en.wikipedia.org/wiki/Pyramid_(image_processing))

- **criteria:** Providing two `criteria` to perform Lucas-Kanade Optical Flow. `TERM_CRITERIA_COUNT` shows The max number of iterations(10 below) and `TERM_CRITERIA_EPS` represents the epsilon (0.03 below). More iterations means a more exhaustive search, and a smaller epsilon finishes earlier. These are primarily useful in exchanging speed vs accuracy of tracking, but mainly stay the same.

In [3]:
# Parameters for lucas kanade optical flow
lk_params = {
    'winSize': (200, 200),
    'maxLevel': 2,
    'criteria': (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
}

In [4]:
# Capture the video
cap = cv2.VideoCapture()
cap.open("/dev/v4l/by-id/usb-Sonix_Technology_Co.__Ltd._USB_2.0_Camera-video-index0")

# Grab the very first frame of the stream
ret, prev_frame = cap.read()

# Grab a grayscale image (We will refer to this as the previous frame)
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Grabbing the corners
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask = None, **corner_track_params)

# Create a matching mask of the previous frame for drawing on later
mask = np.zeros_like(prev_frame)


while True:
    
    # Grab current frame
    ret,frame = cap.read()
    
    # Grab gray scale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate the Optical Flow on the Gray Scale Frame
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params)
    
    # Using the returned status array (the status output)
    # status output status vector (of unsigned chars); each element of the vector is set to 1 if
    # the flow for the corresponding features has been found, otherwise, it is set to 0.
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]
    
    # Use ravel to get points to draw lines and circles
    for i, (new,prev) in enumerate(zip(good_new, good_prev)):
        
        x_new, y_new = new.ravel()
        x_prev, y_prev = prev.ravel()

        x_new,y_new = int(x_new), int(y_new)
        x_prev,y_prev = int(x_prev), int(y_prev)
        
        
        # Lines will be drawn using the mask created from the first frame
        mask = cv2.line(mask, (x_new,y_new),(x_prev,y_prev), (0,255,0), 3)
        
        # Draw red circles at corner points
        frame = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)
    
    # Display the image along with the mask we drew the line on.
    img = cv2.add(frame,mask)
    cv2.imshow('frame',img)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
   
    # Now update the previous frame and previous points
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)
    
    
cv2.destroyAllWindows()
cap.release()

>**Practical Note:**
<br>- If you move really fast, it may loose some of the keypoints.
<br>- If the keypoints are disappeared, this optical flow will not be albe to track them any further
